# DRPS Analysis
This notebook analyses data scraped from the [DRPS website](http://www.drps.ed.ac.uk/) regarding University of Edinburgh courses. We utilise modified versions of common graph algorithms, specifically for topological sort and connected components.

## Imports
We begin by importing the pandas library for data manipulation. Additionally, we import the `processing.py` and `graphs.py` files. The primary use of `processing.py` is extracting information from raw strings using regular expressions. We implement the `Graph` and `PriorityGraph` classes in `graphs.py`, which utilise adjacency lists. `PriorityGraph` additionally stores a "priority" associated with each vertex, which will be explained later.

In [1]:
import pandas as pd
import processing
from graphs import *

DATA_PATH = "./data/courses.json"
COURSE_CODE_PATTERN = "(math|infr).*" # Modify this RegEx to filter by course codes

courses_df = pd.read_json(DATA_PATH, orient="index").filter(regex=COURSE_CODE_PATTERN, axis="index").dropna()
courses_df.head()

,title,outline_level_year,entry_pre_req,delivery_start
math08073,Undergraduate Course: Mathematics for the Natu...,SCQF Level 8 (Year 1 Undergraduate),,Semester 2
math08074,Undergraduate Course: Engineering Mathematics ...,SCQF Level 8 (Year 1 Undergraduate),,Semester 1
math08075,Undergraduate Course: Engineering Mathematics ...,SCQF Level 8 (Year 1 Undergraduate),,Semester 2
infr11176,Postgraduate Course: Fundamentals of Data Mana...,SCQF Level 11 (Postgraduate),,Semester 1
infr11167,Postgraduate Course: Performance Programming (...,SCQF Level 11 (Postgraduate),It is RECOMMENDED that students have passed\nH...,Semester 2


## Dataset
The dataset initially consists entirely of rows of raw strings, indexed by course codes. We use regular expressions to extract information such as the typical year taken, the semester the course starts in, and which courses are listed as prerequisites. We also define a `start_score` for each course, which is simply `[typical year taken].[starting semster]`. For example, a course typically taken in Year 3 that starts in Semester 2 would be assigned `3.2`. This is a quick-and-dirty way to define a value that we can use as our notion of priority later on.

In [2]:
outline = courses_df["outline_level_year"]
start = courses_df["delivery_start"]

courses_df["year"] = pd.to_numeric(outline.apply(processing.year))
courses_df["start_semester"] = pd.to_numeric(start.apply(processing.semester))

courses_df["start_score"] = courses_df["year"] + courses_df["start_semester"] * 0.1
courses_df = courses_df.dropna()

courses_df["title"] = courses_df["title"].apply(processing.title)

courses_df["prereqs"] = courses_df["entry_pre_req"].apply(processing.prereq)
courses_df.drop(labels=["outline_level_year", "entry_pre_req", "delivery_start"], axis="columns", inplace=True)

courses_df.head()

,title,year,start_semester,start_score,prereqs
math08073,Mathematics for the Natural Sciences 1b,1,2,1.2,[]
math08074,Engineering Mathematics 1a,1,1,1.1,[]
math08075,Engineering Mathematics 1b,1,2,1.2,[]
infr11176,Fundamentals of Data Management,7,1,7.1,[]
infr11167,Performance Programming,7,2,7.2,[infr11175]


Note: We treat all postgraduate courses as being taken in "Year 7", to ensure they come after undegraduate courses, some of which may go up to Year 6 (e.g. in the Edinburgh Medical School).

## Building a Graph
We begin by populating a graph with vertices and edges, where vertices are courses and an edge $(u, v)$ indicates that $u$ is a prerequisite for $v$. Note that graphs as implemented in `graphs.py` are only concerned with abstract structure. The information associated with each vertex (i.e. the course code) we store in a separate dictionary.

In [3]:
G = PriorityGraph()

course_to_vertex = {}
vertex_to_course = {}

for course, _ in courses_df["prereqs"].items():
    i = G.addVertex(courses_df["start_score"][course])
    course_to_vertex[course] = i
    vertex_to_course[i] = course

for course, prereq in courses_df["prereqs"].items():
    for pre_course in prereq:
        if pre_course in course_to_vertex.keys():
            G.addEdge(course_to_vertex[pre_course], course_to_vertex[course])

## Modified Topological Sort
The dataset of courses and their prerequisites, as one would expect, forms a directed acyclic graph. This means that we can define a topological order on the set of courses. To also consider the time when a course is typically taken (since we would want our topological order to list Year 1 courses before Year 4 courses), we also consider the priority of each course, which is the `start_score` we defined earlier. We, at any step where `topSort` can freely choose from a list of vertices which to consider next, use a heap to always choose with respect to priority.

In [4]:
L = G.topSort()
course_order = processing.list_order(map(lambda i : courses_df.loc[vertex_to_course[i]]["title"], L))
course_order[:50]

['Fundamentals of Algebra and Calculus',
 'Introductory Mathematics with Applications',
 'Mathematics for the Natural Sciences 1a',
 'Introduction to Data Science',
 'Engineering Mathematics 1a',
 'Informatics 1 - Introduction to Computation',
 'Introduction to Linear Algebra',
 'Engineering Mathematics 1b',
 'Informatics 1 - Cognitive Science',
 'Mathematics for the Natural Sciences 1b',
 'Proofs and Problem Solving',
 'Informatics 1 - Object Oriented Programming',
 'Calculus and its Applications',
 'Informatics 2C - Introduction to Computer Systems',
 'Accelerated Proofs and Problem Solving',
 'Facets of Mathematics',
 'Informatics Experiential Learning (Level 8)',
 'Discrete Mathematics and Probability',
 'Informatics 2 - Introduction to Algorithms and Data Structures',
 'Accelerated Algebra and Calculus for Direct Entry',
 'Several Variable Calculus and Differential Equations',
 'Probability',
 'Informatics 2 - Foundations of Data Science',
 'Informatics 2D - Reasoning and Agents',

In [5]:
G = Graph()

course_to_vertex = {}
vertex_to_course = {}

for course, _ in courses_df["prereqs"].items():
    i = G.addVertex()
    course_to_vertex[course] = i
    vertex_to_course[i] = course

for course, prereq in courses_df["prereqs"].items():
    for pre_course in prereq:
        if pre_course in course_to_vertex.keys():
            # We want this graph to be undirected
            G.addEdge(course_to_vertex[pre_course], course_to_vertex[course])
            G.addEdge(course_to_vertex[course], course_to_vertex[pre_course])

In [6]:
CC = G.connComp()
CC = sorted(CC, key=lambda l : len(l), reverse=True)

components = pd.Series()

for i, comp in enumerate(CC):
    for v in comp:
        components[vertex_to_course[v]] = i

courses_df["component"] = components
courses_df.head()

,title,year,start_semester,start_score,prereqs,component
math08073,Mathematics for the Natural Sciences 1b,1,2,1.2,[],7
math08074,Engineering Mathematics 1a,1,1,1.1,[],8
math08075,Engineering Mathematics 1b,1,2,1.2,[],9
infr11176,Fundamentals of Data Management,7,1,7.1,[],10
infr11167,Performance Programming,7,2,7.2,[infr11175],11


In [7]:
courses_df[courses_df["component"] == 0]

,title,year,start_semester,start_score,prereqs,component
infr08031,Discrete Mathematics and Probability,2,1,2.1,"[math08057, infr08025, math08058]",0
infr08025,Informatics 1 - Introduction to Computation,1,1,1.1,[],0
infr08029,Informatics 1 - Object Oriented Programming,1,2,1.2,[infr08025],0
infr08030,Informatics 2 - Foundations of Data Science,2,1,2.1,"[infr08025, infr08029, math08057, math08058]",0
infr08026,Informatics 2 - Introduction to Algorithms and...,2,1,2.1,"[infr08025, infr08029, math08059, infr08031]",0
...,...,...,...,...,...,...
math11238,Targeted Causal Learning,7,2,7.2,"[math10093, math11176, infr11130, math11242]",0
math11131,Time Series,7,2,7.2,[math10095],0
math11183,Topics in Applied Operational Research,7,2,7.2,"[math10073, math10065, math11007, math11111]",0
math11227,Topics in Mathematical Physics A,5,2,5.2,"[math11235, math11138]",0
